In [3]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(open('sancionados.html', 'rb').read())

In [26]:
def process_row(row):
    output = {'name':row.text}
    try:
        text = row.find('a')['onclick']
        output['link']= text.split("'")[1]
        
    except:
        output['link'] = 'None'
    return output

In [27]:
process_row(soup.find_all('tr')[4])

{'name': ' 3R INNOVACIÓN Y DISEÑO, S.A. DE  C.V. ', 'link': '006410041/2013'}

In [101]:
empresas = [process_row(row) for row in soup.find_all('tr')[4:]]

In [31]:
import requests

In [100]:
def is_style(x):
    return 'color' in x or 'font' in x or 'px}' in x

In [36]:
def get_file_data(file_num):
    url = 'https://directoriosancionados.funcionpublica.gob.mx/SanFicTec/jsp/Ficha_Tecnica/FichaSinTabla.jsp'
    params = {'expe':file_num,
              'tipo':1,
              'persona':3}
    return requests.post(url, data = params).content

In [99]:
def process_file_data(body):
    page_data = BeautifulSoup(body.decode(encoding='latin-1')).text
    page_data = page_data.replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n')
    interest_lines = [x for x in page_data.split('\n') if ':' in x]
    remove_style = [x for x in interest_lines if not is_style(x) ]
    key_value_data = [x.split(':') for i,x in enumerate(remove_style) if i < len(remove_style)-2 ]
    return {x[0]: x[1] for x in key_value_data}

In [108]:
output_data = []
for i,row in enumerate(empresas):
    if i%100 == 0: print(i)
    output_row = row.copy()
    body = get_file_data(output_row['link'])
    file_data = process_file_data(body)
    output_row.update(file_data)
    output_data.append(output_row)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [109]:
import pickle

In [110]:
import json

In [112]:
json.dump(output_data, open('empresas.json', 'w'))

In [114]:
import pandas as pd

/Users/ricalanis/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [117]:
pd.DataFrame(output_data)

,name,link,Infractor,Núm. Exp.,Fecha Notificación de Resolución,Monto de la Multa,Objeto Social o del Contrato,Causa,OIC Responsable,Responsable de la información,Cargo,Teléfono,Correo electrónico,Publicación en el DOF,Plazo de inhabilitación,Inicia,Termina,Observaciones
0,"3R INNOVACIÓN Y DISEÑO, S.A. DE C.V.",006410041/2013,"3R INNOVACIÓN Y DISEÑO, S.A. DE C.V.",0041/2013,19 DE AGOSTO DE 2014,"$ 90,664.00",3RINNOVACION@LIVE.COM.MX.,LA EMPRESA POR CAUSAS IMPUTABLES A ELLA NO FO...,\r,LIC. JORGE PERALTA PORRAS,TITULAR DEL ÁREA DE RESPONSABILIDADES,52382700,jorge.peraltap@imss.gob.mx,NaN,NaN,NaN,NaN,NaN
1,4PRESS SA DE CV,111370002/2019,4PRESS SA DE CV,0002/2019,14 DE ENERO DE 2020,"$ 154,020.00",IMPRESIÓN DE FORMAS CONTINUAS Y OTROS IMPRESOS,INCUMPLIMIENTO AL ARTICULO 59 SEGUNDO PÁRRAFO...,\r,EDGAR ULISES VERGARA SORIA,TITULAR DEL ÁREA DE RESPONSABILIDADES,54810400,edgar_vergara@conaliteg.gob.mx,NaN,NaN,NaN,NaN,NaN
2,A AND P INTERNATIONAL,185770002/2016,A AND P INTERNATIONAL,0002/2016,06 DE DICIEMBRE DE 2016,$ 1.00,NaN,"LO PREVISTO EN EL NUMERAL 59, FRACCIÓN III, D...",\r,NaN,NaN,NaN,NaN,27 DE DICIEMBRE DE 2016,4 AÑOS,28 DE DICIEMBRE DE 2016,28 DE DICIEMBRE DE 2020,NaN
3,"A Y M CONSTRUCTORA, S.A. DE C.V.",000090121/2006,"A Y M CONSTRUCTORA, S.A. DE C.V.",0121/2006,30 DE MARZO DE 2007,"$ 72,540.00",NaN,PRESENTÓ INFORMACIÓN FALSA RESPECTO A ENCONTR...,\r,ADRIÁN ONTIVEROS LÓPEZ,TITULAR DEL ÁREA DE RESPONSABILIDADES,5557239300,a.ontiveros.oic@funcionpublica.gob.mx,05 DE ABRIL DE 2007,3 MESES,06 DE ABRIL DE 2007,06 DE JULIO DE 2007,NaN
4,"A Y V INDUSTRIAL Y COMERCIAL, S.A. DE C.V.",184000001/2013,"A Y V INDUSTRIAL Y COMERCIAL, S.A. DE C.V.",0001/2013,08 DE MAYO DE 2014,"$ 133,502.74","DISEÑAR, PROYECTAR, PLANEAR, ORGANIZAR, ASESO...",PRESENTACION DE DOCUMENTACION E INFORMACION F...,\r,EDUARDO DE LA LUZ GARCÍA,DIRECTOR GENERAL ADJUNTO DE SANCIONES,2056,edelaluz@funcionpublica.gob.mx,20 DE MAYO DE 2014,3 MESES,21 DE MAYO DE 2014,21 DE AGOSTO DE 2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,"ZAMPA, S.A DE C.V.",185720011/2012,"ZAMPA, S.A DE C.V.",0011/2012,09 DE ENERO DE 2013,"$ 93,495.00",CONSULTORÍA EN PLANEACIÓN ESTRATÉGIACA DE NEG...,"NO FORMALIZACIÓN DE CONTRATO , NO ACREDITÓ ES...",\r,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LA SOCIEDAD PROMOVIO JUICIO DE NULIDAD, LA PR..."
2035,ZEFERINO ROMERO PEÑA,006370012/2010,ZEFERINO ROMERO PEÑA,0012/2010,07 DE JUNIO DE 2011,"$ 84,940.00",REALIZAR TRABAJOS DE CONTROL DE LAS OBRAS ORI...,EN EL PROCEDIMIENTO DE CONTRATACIÓN PRESUNTAM...,\r,KARLA MARÍA GONZÁLEZ SALCEDO,TITULAR DEL ÁREA DE RESPONSABILIDADES,54816800 - 26508,karla.gonzalezs@issste.gob.mx,15 DE JUNIO DE 2011,3 MESES,16 DE JUNIO DE 2011,16 DE SEPTIEMBRE DE 2011,NaN
2036,"ZU GAS, S.A. DE C.V.",000270102/2018,"ZU GAS, S.A. DE C.V.",0102/2018,18 DE OCTUBRE DE 2019,"$ 905,696.00",NaN,"PARTICIPÓ EN EL PROCEDIMIENTO, TENIENDO LOS M...",\r,MARÍA GUADALUPE VARGAS ÁLVAREZ,DIRECTORA GENERAL,2086,maria.vargas@funcionpublica.gob.mx,05 DE NOVIEMBRE DE 2019,2 AÑOS,06 DE NOVIEMBRE DE 2019,06 DE NOVIEMBRE DE 2021,NaN
2037,"ZZO CONSTRUCCIONES, S.A. DE C.V.",000200014/2014,"ZZO CONSTRUCCIONES, S.A. DE C.V.",0014/2014,24 DE ABRIL DE 2015,"$ 1,869,900.00",CONSTRUCCIÓN DE TODA CLASE DE INGENIERÍA CIVI...,"LA EMPRESA ZZO CONSTRUCCIONES, S.A. DE C.V. P...",\r,NaN,NaN,NaN,NaN,25 DE MAYO DE 2015,3 AÑOS,26 DE MAYO DE 2015,26 DE MAYO DE 2018,NaN
